# Tutorial 2: Charged Systems: Counterion Condensation

## Introduction

In this tutorial, we simulate a charged system consisting of a fixed charged rod with ions around it. This setup represents a simplified model for polyelectrolyte gels. We will investigate the condensation of ions onto the oppositely charged rod and compare the results to a meanfield analytical solution obtained from Poisson-Boltzmann (PB) theory.
Finally we will go beyond the expected applicability of PB and add additional salt ions to observe an overcharging effect.

In general the tutorial follows "Deserno, Markus, Christian Holm, and Sylvio May. "Fraction of condensed counterions around a charged rod: Comparison of Poisson− Boltzmann theory and computer simulations. Macromolecules 33.1 (2000): 199-206.", se we refer to that publication for further reading.

In [ ]:
from espressomd import System, interactions, electrostatics

import numpy as np
from scipy import optimize
%matplotlib inline
import matplotlib.pyplot as plt


# System setup

After importing the necessary **ESPResSo** features and external modules, we define the cubical system geometry and some physical parameters (which define our unit system).

In [ ]:

# system parameters
rod_length = 24.
bjerrum_length = 1.0
kT = 1.
# we assume a unit system where the elementary charge is 1

system = System(box_l=3*[rod_length])
system.time_step = 0.01
system.cell_system.skin = 0.4

We will build the charged rod from individual particles that are fixed in space. With this, we can use the particle-based electrostatics methods of **ESPResSo**. For analysis, we give the rod particles a different type than the counterions.

In [ ]:
# interaction parameters
wca_epsilon = 1.0
ion_diameter = 1.0
rod_radius = 1.0
# particle types 
rod_type = 1
counterion_type = 2

# Exercise:
* Setup the purely repulsive Weeks-Chandler-Anderson (WCA) interaction [TODO: link docu] between the ions and between the ions and the rod particles. Use the parameters introduced in the cell above.

```python
#ion-ion interaction
system.non_bonded_inter[counterion_type,counterion_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter)

# ion-rod interaction
system.non_bonded_inter[counterion_type,rod_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)
```

Now we need to place the particles in the box

# Exercise
* implement a function to place the rod particles along the $x_3$ axis in the middle of the simulation box and the ions randomly distributed 
* Use the signature ``setup_rod_and_counterions(system, ion_valency, counterion_type, rod_charge_dens, N_rod_beads, rod_type) ``
* determine the number of counterions from the condition of neutrality for the whole system (the rod should be positive, the counterions negative)
* assign the rod particles and cointerions their correct ``type``
* Give the counterions a charge according to their ``ion_valency``
* Give the rod particles a charge such that the ``rod_charge_dens`` is uniformely distributed along the ``N_rod_beads`` individual particles
* Fix the rod particles in space so they do not get moved if forces act upon them

# Hint
* Look into [TODO:link espresso particle properties] to find the keywords to set charges and fix particles

```python
def setup_rod_and_ions(system, ion_valency, counterion_type,
                 rod_charge_dens, N_rod_beads, rod_type):
    
    # calculate charge of the single rod beads
    total_rod_charge = rod_charge_dens*system.box_l[0]
    rod_charge_per_bead = total_rod_charge/N_rod_beads
    
    # number of counterions
    N_ions = int(total_rod_charge/ion_valency)
    
    assert(abs((total_rod_charge-N_ions*ion_valency)/total_rod_charge)<1e-8)
    
    for idx in range(N_rod_beads):
        system.part.add(pos=[rod_length/2.,rod_length/2.,idx/N_rod_beads], 
                        type=rod_type, q=rod_charge_per_bead, fix=3*[True])

    for _ in range(N_ions):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=counterion_type, q=-ion_valency)
```

```python
def setup_rod_and_counterions(system, ion_valency, counterion_type,
                 rod_charge_dens, N_rod_beads, rod_type):
    
    # calculate charge of the single rod beads
    total_rod_charge = rod_charge_dens*system.box_l[0]
    rod_charge_per_bead = total_rod_charge/N_rod_beads
    
    # number of counterions
    N_ions = int(total_rod_charge/ion_valency)
    
    for idx in range(N_rod_beads):
        system.part.add(pos=[rod_length/2.,rod_length/2.,idx/N_rod_beads], 
                        type=rod_type, q=rod_charge_per_bead, fix=3*[True])

    for _ in range(N_ions):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=counterion_type, q=-ion_valency)
```

In [ ]:
counterion_valency = 1
rod_charge_dens = 1

# number of beads that make up the rod
N_rod_beads = 50

setup_rod_and_counterions(system, counterion_valency, counterion_type,
             rod_charge_dens, N_rod_beads, rod_type)

#check that the particle setup was done correctly
assert(sum(system.part[:].q)==0)
assert(np.all(system.part.select(type=rod_type).fix))

Now we set up the electrostatics method to calculate the forces and energies from the longrange coulomb interaction. **ESPResSo** uses so-called <tt>actors</tt> for electrostatics, magnetostatics and hydrodynamics. This ensures that unphysical combinations of algorithms are avoided, for example simultaneous usage of two electrostatic interactions. Adding an actor to the system also activates the method and calls necessary initialization routines. Here, we define a P$^3$M object with parameters Bjerrum length and rms force error. This automatically starts a tuning function which tries to find optimal parameters for P$^3$M and prints them to the screen.

In [ ]:
p3m_params = {'prefactor':kT*bjerrum_length,
              'accuracy':1e-3}
p3m = electrostatics.P3M(**p3m_params)
system.actors.add(p3m)

Before we can start the simulation, we need to remove the overlap between particles to avoid large forces which would crash the simulation. For this, we use the steepest descent integrator with a relative convergence criterium for forces and energies.

In [ ]:
def remove_overlap(system, sd_params):   
    #remove overlap by steepest descent
    # Set up steepest descent integration
    system.integrator.set_steepest_descent(f_max=0,
                                           gamma=sd_params['damping'],
                                           max_displacement=sd_params['max_displacement'])
    
    # Initialize integrator to obtain initial forces
    system.integrator.run(0)
    maxforce = np.max(np.linalg.norm(system.part[:].f, axis = 1))
    energy = system.analysis.energy()['total']
    
    i = 0
    while i < sd_params['max_steps']//sd_params['emstep']:
        prev_maxforce = maxforce
        prev_energy = energy
        print(prev_energy)
        system.integrator.run(sd_params['emstep'])
        maxforce = np.max(np.linalg.norm(system.part[:].f, axis = 1))
        relforce = np.abs((maxforce-prev_maxforce)/prev_maxforce)
        energy = system.analysis.energy()['total']
        relener = np.abs((energy-prev_energy)/prev_energy)
        print("minimization step: {:4.0f}\tmax. rel. force change:{:+3.3e}\trel. energy change:{:+3.3e}".format((i+1)*sd_params['emstep'],relforce, relener))
        if relforce < sd_params['f_tol'] or relener < sd_params['e_tol']:
            break
        i += 1
        
    system.integrator.set_vv()

In [ ]:
#steepest descent params
STEEPEST_DESCENT_PARAMS = {'f_tol':1e-2,
                          'e_tol':1e-5,
                          'damping':30,
                          'max_steps':10000,
                          'max_displacement':0.01,
                          'emstep':10}

remove_overlap(system,STEEPEST_DESCENT_PARAMS)

After the overlap is removed, we activate a thermostat to simulate the system at a given temperature.

In [ ]:
langevin_params = {'kT':kT,
                   'gamma':0.5,
                   'seed':42}
system.thermostat.set_langevin(**langevin_params)

Now we are ready to start the integration loop. As we are interested in the condensation of counterions on the rod, the physical quantity of interest is the density $\rho(r)$ of charges around the rod, where $r$ is the distance from the rod. We need many samples (frames) to calculate the density from histograms.
The samples should be taken from an equilibrated system and not be correlated to each other.

# Exercise
* Write a function to integrate the system and return the total energy and the radial distance of the ions from the rod
* Use the signature ``integrate_calc_observables(system, N_frames, steps_per_frame, ion_types)``
* ``N_frames`` is the total number of samples taken from the system
* ``steps_per_frame`` is the the number of integration steps between consecutive frames
* ``ion_types`` is a list of types for which the radial distances should be calculated. For the moment we only have counterions, but later we will also add additional salt ions for which we would like to calculate the density
* return a list of the total system energy as well as a dictionary of the radial positions sorted by ion type, e.g. ``radial_distances[counterion_type] = [1.1,4.5,2.7,...]``

# Hints
* We are only interested in the distance from the rod, so the $x_3$ coordinate is irrelevant
* Use ``system.part.select(type=...)`` to get only the particles of a specific type
* The system energy can be calculated using **ESPReSso**'s analysis functions [TODO: link], the radial distance has to be computed manually

```python
def integrate_calc_observables(system, N_frames, steps_per_frame, ion_types):
    energies = []
    radial_distances = []
    
    particles_by_type = {}
    radial_distances = {}
    for ion_type in ion_types:
        particles_by_type[ion_type] = system.part.select(type=ion_type)
        radial_distances[ion_type] = []
    
    system_center = np.array(system.box_l)/2.
    
    for _ in range(N_frames):
        # run run the simulation for a few steps
        system.integrator.run(steps_per_frame)

        energies.append(system.analysis.energy()['total'])
        
        for ion_type, ions in particles_by_type.items():
            for ion in ions:
                radial_distances[ion_type].append(np.linalg.norm(ion.pos_folded[0:2]-system_center[0:2]))
    
    return energies, radial_distances
```



## First runs and system cleanup

Before running the simulations for the histograms, we need to decide how long we need to equilibrate the system and what is a suitable number of integration steps between samples. For this we plot the total energy vs the time steps

In [ ]:
# number of samples taken
N_frames = 100

# number of timesteps per frame
steps_per_frame = 300
energies, _ = integrate_calc_observables(system, N_frames, 
                                       steps_per_frame, [counterion_type])

In [ ]:
ts = np.linspace(0,N_frames*steps_per_frame, num = N_frames)
fig1 = plt.figure()
#plot time in time_steps so we can judge the number of warmup steps and the steps per frame for uncorrelated data
plt.plot(ts, energies)
plt.xlabel('time steps')
plt.ylabel('system total energy')

# Exercise
* estimate a reasonable number of time steps needed to equilibrate the system (``warmup_steps``)
* estimate the scale of the slowest fluctuations to determine how many ``steps_per_frame`` are needed to get uncorrelated samples

```python 
warmup_steps = 5000
steps_per_frame = 1000
```

To run the simulation with different parameters, we need a way to reset the system and return it to an empty state before setting it up again.

# Exercise
* write a function ``clear_system(system)`` that
    * turns off the thermostat
    * removes all particles
    * resets the system clock

```python
def clear_system(system):
    system.thermostat.turn_off()
    system.part.clear()
    system.time = 0.
```

In [ ]:
clear_system(system)

In [ ]:
#TODO replace this function by more intuitive behaviour of p3m.tune()
def retune_p3m(p3m):
    #reset p3m tunable parameters to default to force a retune
    p3m_default_params = p3m.default_params()
    p3m.tune(mesh = p3m_default_params['mesh'],
             cao = p3m_default_params['cao'],
             r_cut = 0,
             #TODO change default value in interface
             alpha = p3m_default_params['alpha'])

Now we are finally ready to run the simulations and produce the data we can compare to the Poisson-Boltzmann predictions. First we define the parameters and then loop over the parameters.

In [ ]:
runs = [{'params':{'counterion_valency':2, 'rod_charge_dens':1},
         'distances':None},
        {'params':{'counterion_valency':1, 'rod_charge_dens':2},
         'distances':None}
       ]
N_frames = 200

# Exercise
* run the simulation for the parameters given in the cell above and save the radial distances in the corresponding dictionary for analysis

# Hints
* Don't forget to clear the system once one parameter set is finished and redo the complete setup
* Don't forget to [TODO] tune ``p3m`` after each change of parameters 

In [ ]:
for run in runs:
    setup_rod_and_counterions(system, run['params']['counterion_valency'], counterion_type,
                 run['params']['rod_charge_dens'], N_rod_beads, rod_type)
    retune_p3m(p3m)
    remove_overlap(system, STEEPEST_DESCENT_PARAMS)
    system.thermostat.set_langevin(**langevin_params)
    system.integrator.run(warmup_steps)
    print('setup and warmup done, starting simulation')
    energies, distances = integrate_calc_observables(system, N_frames, 
                                       steps_per_frame, [counterion_type])
    clear_system(system)
    run['distances'] = distances[counterion_type]
    print('simulation for parameters {} done\n'.format(run['params']))

# Question
* Why does the second simulation take much longer than the first one?

The rod charge density is doubled, so the total charge of the counterions needs to be doubled, too. Since their valency is only half of the one in the first run, there will be four times more counterions in the second run

We plot the density of counterions around the rod as a cumulative charge distribution, meaning the integrated probability to find an amount of charge within the radius $r$ [TODO phrasing]

The rod charge density is doubled so the total charge of the counterions is doubled too. Because they have only half the valency, there are four times more counterions in the second run than in the first one

In [ ]:
def calc_cum_hist(values, bins, normalize = True):
    hist, bins = np.histogram(values, bins=log_bins)
    cum_hist = np.cumsum(hist)
    if normalize:
        cum_hist = cum_hist/cum_hist[-1]
    return cum_hist

In [ ]:
log_bins=np.logspace(np.log10(rod_radius+ion_diameter/2.), np.log10(rod_length/2.),num=30)

fig, ax = plt.subplots()
for run in runs:
    cum_hist = calc_cum_hist(run['distances'], log_bins)
    ax.plot(log_bins[1:],cum_hist, label = 'lambda = {}, nu = {}'.format(run['params']['rod_charge_dens'], 
                                                                          run['params']['counterion_valency']))
    
ax.set_xscale('log')
ax.legend()
plt.xlabel('r')
plt.ylabel('P(r)')

In the semilogarithmic plot we see an inflection point of the cumulative charge distribution which is the indicator for ion condensation. To compare to the meanfield approach of PB, we calculate the solution of the analytical expressions given in [link 10.1021/ma990897o]

In [ ]:
def eq_to_solve_for_gamma(gamma, manning_parameter, rod_radius, max_radius):
    #eq 7 - eq 6 from 10.1021/ma990897o
    return gamma*np.log(max_radius/rod_radius) - np.arctan(1/gamma) + np.arctan((1-manning_parameter)/gamma)

def calc_manning_radius(gamma,max_radius):
    #eq 7 from 10.1021/ma990897o
    return max_radius*np.exp(-np.arctan(1./gamma)/gamma)  

def calc_PB_probability(r, manning_parameter, gamma, manning_radius):
    #eq 8 and 9 from 10.1021/ma990897o
    return 1./manning_parameter + gamma/manning_parameter * np.tan(gamma*np.log(r/manning_radius))


For multivalent counterions, the manning parameter $\xi$ has to be multiplied by the valency. The result depends only on the product of ``rod_charge_dens`` and ``ion_velancy``, so we only need one curve

In [ ]:
rod_charge_density = runs[0]['params']['rod_charge_dens']
ion_valency = runs[0]['params']['counterion_valency']
manning_parameter_times_valency = bjerrum_length*rod_charge_density*ion_valency

gamma = optimize.fsolve(eq_to_solve_for_gamma, 1, args = (manning_parameter_times_valency,rod_radius+ion_diameter/2.,rod_length/2.))
manning_radius = calc_manning_radius(gamma, rod_length/2.)

PB_probability = calc_PB_probability(log_bins, manning_parameter_times_valency,gamma, manning_radius)


ax.plot(log_bins, PB_probability, label = f'PB xi*nu = {manning_parameter_times_valency}')
ax.legend()
ax.set_xscale('linear')
fig

We see a good correspondence of the PB solution with the simulation results in the case where the counterion valence is 1, but there are strong deviations if the counterions are more charged. 

# Overcharging by added salt

Above simulations were performed for a system where all ions come from dissociation off the polyelectrolyte. We can also look at systems where there are additional salt ions present.

In [ ]:
def add_salt(system, anion_params, cation_params):        
    for _ in range(anion_params['number']):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=anion_params['type'], q=-anion_params['valency'])
        
    for _ in range(cation_params['number']):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=cation_params['type'], q=cation_params['valency'])
    

In [ ]:
anion_params = {'type':3,
                'valency':2,
                'number':500}
cation_params = {'type':4,
                 'valency':2,
                 'number':500}
rod_length = 62.5

total_anion_charge = -anion_params['number']*anion_params['valency']
total_cation_charge = cation_params['number']*cation_params['valency']
assert( abs(total_anion_charge+ total_cation_charge)/total_cation_charge < 1e-10)

counterion_valency = 1 
rod_charge_dens = 1

all_ion_types = [counterion_type, anion_params['type'],cation_params['type'] ]

#set interactions of salt with the rod and all ions
for salt_type in [anion_params['type'], cation_params['type']]:
    system.non_bonded_inter[salt_type,rod_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)
    for ion_type in all_ion_types:
        system.non_bonded_inter[salt_type,ion_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter)
    



In [ ]:
clear_system(system)
system.box_l = 3*[rod_length]
setup_rod_and_counterions(system, counterion_valency, counterion_type,
                   rod_charge_dens, N_rod_beads, rod_type)
add_salt(system, anion_params, cation_params)
retune_p3m(p3m)
remove_overlap(system, STEEPEST_DESCENT_PARAMS)
system.thermostat.set_langevin(**langevin_params)
system.integrator.run(warmup_steps)
print('setup and warmup done, starting simulation')
energies, distances = integrate_calc_observables(system, N_frames, 
                                   steps_per_frame, all_ion_types)
print('simulation done')


# Exercise
* create cumulative histograms for each type of ion in the system
* Add them together weighted by their valency to get the cumulative charge distribution
* Normalize the distribution by the total ion charge

# Hints
* use ``calc_cum_hist()`` 
* set ``normalize = False`` to account for the fact that there are a different number of ions depending on their type

```python
counterion_hist = calc_cum_hist(distances[counterion_type], log_bins, normalize = False)
anion_hist = calc_cum_hist(distances[anion_params['type']], log_bins, normalize = False)
cation_hist = calc_cum_hist(distances[cation_params['type']], log_bins, normalize = False)

charge_hist = cation_params['valency'] * cation_hist - anion_params['valency'] * anion_hist - counterion_valency * counterion_hist
charge_hist = charge_hist/charge_hist[-1]
```

In [ ]:
fig2, ax2 = plt.subplots()
ax2.plot(log_bins[1:], charge_hist)
ax2.set_xscale('log')
plt.xlabel('r')
plt.ylabel('P(r)')

fig = plt.figure()
plt.plot(energies)
plt.show()